In [1]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

## Working with PDFs with images

PDFs contain more than rich formatting - they have images!

<img src="data/imgs/hai_ai-index-report-2025_chapter2_final_07_of_86.jpg" width="200px" />
<img src="data/imgs/hai_ai-index-report-2025_chapter2_final_32_of_86.jpg" width="200px" />
<img src="data/imgs/hai_ai-index-report-2025_chapter2_final_49_of_86.jpg" width="200px" />

How do we work with these for RAG?

### Approach 1 - Extract text and images separately

Some libraries (like `docling`) can extract text and images from PDFs, and convert them into Markdown files.

In [2]:
from pathlib import Path

data_folder = Path("data/pdfs")
output_dir = Path("data/parsed")
output_dir.mkdir(parents=True, exist_ok=True)

In [3]:
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling_core.types.doc import ImageRefMode

IMAGE_RESOLUTION_SCALE = 2.0


def parse_pdf_with_images(input_doc_path: Path, output_dir: Path):
    # Reference: https://docling-project.github.io/docling/examples/export_figures/
    md_filename = output_dir / f"{input_doc_path.name.split('.')[0]}-parsed-w-imgs.md"
    if md_filename.exists():
        print(f"Skipping {md_filename} as it already exists.")
        return

    pipeline_options = PdfPipelineOptions()
    pipeline_options.images_scale = IMAGE_RESOLUTION_SCALE
    pipeline_options.generate_picture_images = True

    doc_converter = DocumentConverter(
        format_options={
            InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
        }
    )

    conv_res = doc_converter.convert(input_doc_path)

    # Save markdown with embedded pictures
    conv_res.document.save_as_markdown(md_filename, image_mode=ImageRefMode.REFERENCED)


pdf_names = [f.name for f in data_folder.glob("*.pdf") if f.is_file()]

for pdf_fname in pdf_names:
    print(f"Processing file: {pdf_fname}")

    input_doc_path = data_folder / pdf_fname

    print(f"Converting document {input_doc_path} to multimodal pages...")
    parse_pdf_with_images(input_doc_path, output_dir)


Processing file: hai_ai-index-report-2025_chapter2_final.pdf
Converting document data/pdfs/hai_ai-index-report-2025_chapter2_final.pdf to multimodal pages...
Skipping data/parsed/hai_ai-index-report-2025_chapter2_final-parsed-w-imgs.md as it already exists.
Processing file: howto-free-threading-python.pdf
Converting document data/pdfs/howto-free-threading-python.pdf to multimodal pages...
Skipping data/parsed/howto-free-threading-python-parsed-w-imgs.md as it already exists.
Processing file: manual_bosch_WGG254Z0GR.pdf
Converting document data/pdfs/manual_bosch_WGG254Z0GR.pdf to multimodal pages...
Skipping data/parsed/manual_bosch_WGG254Z0GR-parsed-w-imgs.md as it already exists.


In [4]:
md_filepath = Path("data/parsed/manual_bosch_WGG254Z0GR-parsed-w-imgs.md")
md_txt = md_filepath.read_text()
print(md_txt[:1000])

![Image](manual_bosch_WGG254Z0GR-parsed-w-imgs_artifacts/image_000000_d4664b79287c280d55474fa744bafbb6a892977e75d4baf4d92898bff5df14b5.png)

![Image](manual_bosch_WGG254Z0GR-parsed-w-imgs_artifacts/image_000001_6c49944028c7cd7282f76d99434ed8c9aa3dab0af8411b4df8cbb8492adf5279.png)

## Washing machine

## WGG254Z0GR

User manual and installation

[en] instructions

## Futher information and explanations are available online:

![Image](manual_bosch_WGG254Z0GR-parsed-w-imgs_artifacts/image_000002_71ba2e278ecb31d5477d5b97185c8f2215227116e1615b161b0883eeea87bb26.png)

## Table of contents

| 1 Safety...........................................                      | 1 Safety...........................................                        | 4                           |
|--------------------------------------------------------------------------|----------------------------------------------------------------------------|-----------------------------|
| 1.1                                    

#### Chunking text files with images

More complex than just text, since we need to handle images as well.

- Must include entire image string in the chunk
- When vectorizing, optionally include base64 of image
    - Your embedding model must be multimodal

Chunking becomes more complex.

One method: try a specialized library like `chonkie` to handle this

Chonkie offers a variety of chunking strategies:

<img src="assets/chonkie_methods.png" />

There isn't going to be a "one size fits all" solution for chunking PDFs with images. But these libraries can help you get started.

Let's try a couple of different approaches:

In [5]:
from chonkie import RecursiveChunker

# Initialize the recursive chunker to chunk Markdown
chunker = RecursiveChunker.from_recipe("markdown", lang="en")

In [6]:
chunk_texts = chunker.chunk(md_txt)

In [7]:
import textwrap

for chunk in chunk_texts[:5]:
    print(f"\n" + "=" * 40)
    print(f"Token count: {chunk.token_count}")
    print(f"Chunk text:")
    wrapped_text = textwrap.fill(chunk.text[:500]+"...", width=80)
    print(textwrap.indent(wrapped_text, "    "))


Token count: 569
Chunk text:
    ![Image](manual_bosch_WGG254Z0GR-parsed-w-imgs_artifacts/image_000000_d4664b7928
    7c280d55474fa744bafbb6a892977e75d4baf4d92898bff5df14b5.png)
    ![Image](manual_bosch_WGG254Z0GR-parsed-w-imgs_artifacts/image_000001_6c49944028
    c7cd7282f76d99434ed8c9aa3dab0af8411b4df8cbb8492adf5279.png)  ## Washing machine
    ## WGG254Z0GR  User manual and installation  [en] instructions  ## Futher
    information and explanations are available online:
    ![Image](manual_bosch_WGG254Z0GR-parsed-w-imgs_artifacts/image_000002_71...

Token count: 2046
Chunk text:
    ## Table of contents  | 1 Safety...........................................
    | 1 Safety...........................................                        | 4
    | |--------------------------------------------------------------------------|--
    --------------------------------------------------------------------------|-----
    ------------------------| | 1.1
    | General information............

Let's try a "semantic" chunker:

In [8]:
from chonkie import SemanticChunker

# Basic initialization with default parameters
chunker = SemanticChunker(
    embedding_model="minishlab/potion-base-8M",  # Default model
    threshold=0.5,                               # Similarity threshold (0-1) or (1-100) or "auto"
    chunk_size=2048,                              # Maximum tokens per chunk
    min_sentences=1                              # Initial sentences per chunk
)

In [9]:
# Chunk text into `chunk_texts` as we've done before
# BEGIN_SOLUTION
chunk_texts = chunker.chunk(md_txt)
# END_SOLUTION

In [10]:
for chunk in chunk_texts[:5]:
    print(f"\n" + "=" * 40)
    print(f"Token count: {chunk.token_count}")
    print(f"Chunk text:")
    wrapped_text = textwrap.fill(chunk.text[:500]+"...", width=80)
    print(textwrap.indent(wrapped_text, "    "))


Token count: 200
Chunk text:
    ![Image](manual_bosch_WGG254Z0GR-parsed-w-imgs_artifacts/image_000000_d4664b7928
    7c280d55474fa744bafbb6a892977e75d4baf4d92898bff5df14b5.png)
    ![Image](manual_bosch_WGG254Z0GR-parsed-w-imgs_artifacts/image_000001_6c49944028
    c7cd7282f76d99434ed8c9aa3dab0af8411b4df8cbb8492adf5279.png)  ## Washing machine
    ## WGG254Z0GR  User manual and installation  [en] instructions  ## Futher
    information and explanations are available online: ...

Token count: 134
Chunk text:
     ![Image](manual_bosch_WGG254Z0GR-parsed-w-imgs_artifacts/image_000002_71ba2e278
    ecb31d5477d5b97185c8f2215227116e1615b161b0883eeea87bb26.png)  ## Table of
    contents  | 1 Safety........................................... ...

Token count: 1953
Chunk text:
                         | 1 Safety...........................................
    | 4                           | |-----------------------------------------------
    ---------------------------|-----------------------

### Set up Weaviate Collection

In [11]:
import weaviate
import os

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=os.getenv("WEAVIATE_URL"),
    auth_credentials=os.getenv("WEAVIATE_API_KEY"),
    headers={
        "X-Cohere-Api-Key": os.getenv("COHERE_API_KEY"),
    },
)

/Users/jphwang/code/demo-code/202507-lipari/.venv/lib/python3.10/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.0 is exactly one major version older than the runtime version 6.31.1 at v1/aggregate.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/Users/jphwang/code/demo-code/202507-lipari/.venv/lib/python3.10/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.0 is exactly one major version older than the runtime version 6.31.1 at v1/base.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/Users/jphwang/code/demo-code/202507-lipari/.venv/lib/python3.10/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.0 is exactly one major version older than the runtime version 6.31.1 at v1/base_search.proto. Please update the gencode to avo

In [12]:
client.collections.delete("Chunks")

In [13]:
from weaviate.classes.config import Property, DataType, Configure, Tokenization

client.collections.create(
    name="Chunks",
    properties=[
        Property(
            name="document_title",
            data_type=DataType.TEXT,
        ),
        Property(
            name="chunk",
            data_type=DataType.TEXT,
        ),
        Property(
            name="chunk_number",
            data_type=DataType.INT,
        ),
        Property(
            name="filename",
            data_type=DataType.TEXT,
            tokenization=Tokenization.FIELD
        ),
    ],
    vector_config=[
        Configure.Vectors.text2vec_cohere(
            name="default",
            source_properties=["document_title", "chunk"],
            model="embed-v4.0"
        )
    ]
)

In [14]:
chunks = client.collections.get("Chunks")

### Import data

In [15]:
from tqdm import tqdm

with chunks.batch.fixed_size(batch_size=100) as batch:
    for i, chunk_text in tqdm(enumerate(chunk_texts)):
        obj = {
            "document_title": "Bosch WGG254Z0GR Manual",
            "filename": "data/pdfs/manual_bosch_WGG254Z0GR.pdf",
            "chunk": chunk_text.text,
            "chunk_number": i + 1,
        }

        # Add object to batch for import with (batch.add_object())
        # BEGIN_SOLUTION
        batch.add_object(
            properties=obj
        )
        # END_SOLUTION

171it [00:00, 60679.02it/s]
{'message': 'Failed to send all objects in a batch of 71', 'error': 'WeaviateInsertManyAllFailedError(\'Every object failed during insertion. Here is the set of all errors: send POST request: Post "https://api.cohere.ai/v2/embed": context deadline exceeded (Client.Timeout exceeded while awaiting headers)\')'}
{'message': 'Failed to send 71 objects in a batch of 71. Please inspect client.batch.failed_objects or collection.batch.failed_objects for the failed objects.'}


### RAG queries

How do we perform RAG in this scenario? 

This is a bit different, because we haven't embedded the images (or stored them in Weaviate).

In this scenario, let's:

- Retrieve text chunks
- Get images referred to in the text
- Convert the images to base64
- Send (retrieved text + images + prompt) to LLM for RAG

In [16]:
response = chunks.query.hybrid(
    query="How to clean the drain pump",
    limit=10
)

for o in response.objects:
    print(f"\n" + "=" * 40)
    print(o.properties["chunk"][:1000] + "...")



![Image](manual_bosch_WGG254Z0GR-parsed-w-imgs_artifacts/image_000042_346828476ca0377e11aa024cf725f4fa099db6b690678ba9c0b9ffdaa6c1e5c8.png)

On certain models, specific details such as the colour and shape may differ from those pictured.

- 2 Maintenance flap for the drain pump → Page 36
...

| Καθαρισμός κάδου (Drum Clean)             | Clean and care for the drum. Use this programme in the following cases: ¡ Before using the appliance for the first time ¡ If you regularly wash at temperatures of 40 °C or lower ¡ After a prolonged period of non-use Use washing powder that contains bleach. Add the washing powder containing bleach to compartment II for the main wash. → "Detergent drawer", Page 17 Halve the amount of detergent to avoid foaming.                                                                                                                                                                                                                                                      

In [17]:
import re

def extract_image_paths(text):
    """Extract image paths from markdown-style image references."""
    pattern = r'!\[.*?\]\((.*?)\)'
    return re.findall(pattern, text)

In [18]:
def get_image_base64s(image_paths, base_path=None):
    import base64
    base64_images = []
    for img_path in image_paths:
        full_path = Path(base_path) / img_path if base_path else Path(img_path)
        image_bytes = full_path.read_bytes()
        base64_string = base64.b64encode(image_bytes).decode("utf-8")
        base64_images.append(base64_string)

    return base64_images

In [19]:
all_chunks = ""
all_images = []

for o in response.objects:
    chunk_text = o.properties["chunk"]
    image_paths = extract_image_paths(chunk_text)
    all_images.extend(get_image_base64s(image_paths, base_path="data/parsed"))

    all_chunks += "\n\n" + chunk_text

In [20]:
task_text = """
How do I clean the drain pump? Answer based on the provided text and images.

Describe the details from the figures as well, if necessary.
""" + "\n\n" + all_chunks

message = {
    "role": "user", "content": [
        {"type": "text", "text": task_text}
    ]
}

for img in all_images:
    content = {
        "type": "image",
        "source": {
            "type": "base64",
            "media_type": "image/png",
            "data": img,
        }
    }
    # Append `content`` to message["content"]
    # BEGIN_SOLUTION
    message["content"].append(content)
    # END_SOLUTION

In [21]:
import anthropic

anthropic_response = anthropic.Anthropic().messages.create(
    model="claude-sonnet-4-20250514",
    max_tokens=1024,
    # Add [message] as the messages to pass to Claude
    # BEGIN_SOLUTION
    messages=[message]
    # END_SOLUTION
)

In [22]:
print(anthropic_response.content[0].text)

Based on the provided text and images, here's how to clean the drain pump:

## Location of the Drain Pump
The drain pump can be accessed through the **maintenance flap** (labeled as item 2 in the first image), which is located at the bottom front of the washing machine.

## Cleaning Steps
The text references that detailed instructions for cleaning the drain pump can be found on "Page 36" of the manual, but those specific step-by-step instructions are not included in the provided excerpt.

## Important Safety Information
Before cleaning the drain pump, note these safety precautions mentioned in the text:

- **Switch off the appliance** and unplug it from the mains
- **Turn off the water tap**
- Be aware that **when draining, the water outlet hose is subject to water pressure** and may come loose
- **Secure the water outlet hose** against unintentional loosening
- Never use **steam or high-pressure cleaners** on the appliance

## What the Images Show
- **First image**: Shows the front vi

In [23]:
client.close()